# 결측치?

```
세상의 모든 데이터는 완벽하지 않다. 그래서 누락된 값, 결측치(Missing Value)가 존재한다. 데이터서는 이 값을 None, NaN으로 표현한다. 이러한 결측치는 데이터 기반으로 머신러닝 모델 학습 품질에 큰 영향을 주는 값이된다. Python 대표적인 머신러닝 모듈인 Scikit-learn estimator 같은 일부 알고리즘은 데이터로 사용하는 모든 값이 의미가 있다는 가정을 하고 있기 때문이다. 그래서 모델링 이전에 이 결측치를 어떻게 처리할지 정하는게 중요하다.

데이터는 다면적인 데이터셋과 시계열 데이터 크게 두 가지로 볼 수 있다. 전처리 단계에서 각 데이터셋의 결측치 처리 방법에 대해서 정리해보자.
```

# 단면 데이터 셋

### 처리 방법
- 방치
- 제거
- Mean / Medain
- Frequent / Zero / Constant
- KNN
- MICE


#### 방치 
```
결측치 처리에서 가장 쉬운 방벙이다. 모델링 과정에서 알고리즘에서 결측치를 무시해주는 파라미터를 넣어 처리할 수 있다. 하지만 이 기능이 없는 경우 모델이 쓰레기가 된다.
```

#### 제거
```
결측치를 가진 데이터를 제거하는 방법이다. 결측치가 가진 하나의 데이터를 제거할 수 있고 결측치를 포함한 피처를 제거할 수 있다. 단, 이 방법은 중요한 정보를 가진 데이터가 제거될 위험성이 있다.
```

In [50]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    0 : np.random.randint(0, 5, 5),
    1 : np.random.randint(0, 5, 5),
    2 : np.random.randint(0, 5, 5)
})

for _ in range(3):
    col = np.random.randint(2)
    row = np.random.randint(5)
    df.loc[row, col] = np.NAN

df

,0,1,2
0,0.0,3.0,0
1,4.0,NaN,1
2,0.0,NaN,0
3,NaN,1.0,0
4,0.0,2.0,2


In [51]:
df.dropna()

,0,1,2
0,0.0,3.0,0
4,0.0,2.0,2


#### Mean / Median

```
결측치를 다른 값들의 평균 혹은 중앙값으로 대체하는 방법.
이 방법은 쉽고 빠르게 결측치를 처리할 수 있고 숫자 범위가 작은 데이터 셋에서 잘 작동한다는 장점이 있다.
단점은 다른 피처와 상관을 고려하지 않는다. 인코딩된 범주형 변수에 사용할 수 없다. 정확성이 떨어진다.
```

In [52]:
df

,0,1,2
0,0.0,3.0,0
1,4.0,NaN,1
2,0.0,NaN,0
3,NaN,1.0,0
4,0.0,2.0,2


In [53]:
# Mean
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean')
df_impute = pd.DataFrame(imp_mean.fit_transform(df))

df_impute

,0,1,2
0,0.0,3.0,0.0
1,4.0,2.0,1.0
2,0.0,2.0,0.0
3,1.0,1.0,0.0
4,0.0,2.0,2.0


In [54]:
# Median
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='median')
df_impute = pd.DataFrame(imp_mean.fit_transform(df))

df_impute

,0,1,2
0,0.0,3.0,0.0
1,4.0,2.0,1.0
2,0.0,2.0,0.0
3,0.0,1.0,0.0
4,0.0,2.0,2.0


#### Frequent / Zero / Constant
```
Frequent : 가장 많이 나왔던 값으로 대체한다. 범주형 변수에서도 사용이 가능하다.
Zero  : 결측치를 0으로 대체한다.
Constant : 결측치를 처리할 값을 직접 지정한다.

이 방법 역시 빠르게 처리가 가능하다. 평균과 중앙값과 다른점은 범주형 변수에서도 사용이 가능하다는 점. 단점은 역시 피처간 상관을 고려되지 않음.
```

In [55]:
df

,0,1,2
0,0.0,3.0,0
1,4.0,NaN,1
2,0.0,NaN,0
3,NaN,1.0,0
4,0.0,2.0,2


In [56]:
# Frequency
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='most_frequent')
df_impute = pd.DataFrame(imp_mean.fit_transform(df))

df_impute

,0,1,2
0,0.0,3.0,0.0
1,4.0,1.0,1.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,2.0,2.0


In [57]:
# Zero
df.fillna(0) # 0을 수정하면 Constant

,0,1,2
0,0.0,3.0,0
1,4.0,0.0,1
2,0.0,0.0,0
3,0.0,1.0,0
4,0.0,2.0,2


#### KNN
```
KNN(K-nearest neighbours) 피처 특성을 이용해 가장 닮은 k개의 데이터를 찾아서 그룹화한다.

장점 : mean, median, frequent 보다 성능 좋다. (데이터셋에 따라 좋지 못할 수 있음)
단점 : 이상치(outlier)에 민감하다.
```

In [58]:
df

,0,1,2
0,0.0,3.0,0
1,4.0,NaN,1
2,0.0,NaN,0
3,NaN,1.0,0
4,0.0,2.0,2


In [59]:
# !pip install impyute

In [60]:
from impyute.imputation.cs import fast_knn
df_impute = pd.DataFrame(fast_knn(df.values, k =2))

df_impute


,0,1,2
0,0.0,3.000000,0.0
1,4.0,1.554201,1.0
2,0.0,1.828427,0.0
3,0.0,1.000000,0.0
4,0.0,2.000000,2.0


#### MICE
```
MICE(Multiple Implutation by Chained Equation) 방식은 다중대체 하는 방법으로 굉장히 복잡한 방법으로 결측값을 다룬다.
1. 결측치가 포함된 원 데이터에서 결측치를 유사값들로 대체하여 데이터셋을 만들어낸다.
2. 이렇게 만들어진 데이터 셋들을 각각 통계모델로 분석해 최종적으로 하나의 데이터셋을 만들어낸다.
```
![image.png](https://blog.kakaocdn.net/dn/dmJPLq/btqGxPRY6Ji/lovcHXD2ef5PDDog0uG2A1/img.png)

In [61]:
df

,0,1,2
0,0.0,3.0,0
1,4.0,NaN,1
2,0.0,NaN,0
3,NaN,1.0,0
4,0.0,2.0,2


In [62]:
from impyute.imputation.cs import mice
df_imputed = pd.DataFrame(mice(df.values))

df_imputed

,0,1,2
0,0.000000,3.0,0.0
1,4.000000,-0.5,1.0
2,0.000000,3.0,0.0
3,2.666667,1.0,0.0
4,0.000000,2.0,2.0


# 시게열데이터의 결측치
```
시계열 데이터에서는 결측치를 제거하게 되는 경우 그 시점의 분산과 평균의 왜곡을 가져온다. 시계열 데이터에서 제거해도 전형 영향이 없는 데이터면 괜찮지만 그렇지 않은 경우 분석결과에 직결되어 치명적인 영향을 줄 수 있다. 이러한 시계열 데이터를 처리하는 방법으로
- LOCF(Last observation carried Forward) : 직전 관측값으로 대체
- NOCB(Next observation carried Backward) : 직후 관측값으로 대체
- Moving Average / Median : 직전 N window의 (평균 / 중앙값)으로 대체

3가지 방법으로 처리하기 어려운 경우가 있다. 결측치가 변곡점이 포함됐을 경우다. 이 경우 보간법(interpolation)을 사용한다.
```
### 보간법 : 기존에 알고있는 특정 지점이나 지역의 속성값을 이용하여 알고자 하는 지점 또는 지역의 속성값을 찾아내는 방법.
```
보간법의 종류에는 선형, 다항식, 스플라인, 지수, 로그, 라그랑지, 뉴튼, 2차원, 공간, 크리깅 등이 있다. 이번에는 선형, 비선형, 스플라인 3가지를 다룬다.

- 선형 보간법
: 가장 간단한 방법으로 알고자 하는 함수가 직선이라는 가정하고 함수값을 추정. 단순히 알려진 데이터를 선으로 이어주면 된다. 가장 간단하고 빠르지만 정확성은 좋지 못하다.

- 다항함수 보간법
: 주어진 점들을 지나는 다항식을 찾는 보간법

- 스플라인 보간법
: 각 구간에서 낮은 차원의 다항식을 사용한다. 앞, 뒤 구간의 다항식들과 자연스럽게 연결될 수 있는 것으로 선택. 각 점에서 앞, 뒤 스플라인 함수가 미분이 가능해야하고 곡률도 같아야 한다.

```
